In [3]:
import win32com.client
import datetime

# 期間指定
weeks_before = 1
weeks_ahead = 8
# 期間：自
st = datetime.date.today() - datetime.timedelta(weeks=weeks_before)
# st = datetime.date(2020,7,1)
# 期間：至
ed = datetime.date.today() + datetime.timedelta(weeks=weeks_ahead)

# Outlookの予定アクセス用のAPI
app = win32com.client.Dispatch("Outlook.Application")
stores = app.Session.Stores

root = app.Session.DefaultStore.GetRootFolder()
ns = app.GetNamespace("MAPI")
cal = ns.GetDefaultFolder(9)

# 期間指定用のフィルタ文字列
filterStr = '[Start]>="{0}" and [Start]<"{1}"'.format(
    st.strftime("%Y/%m/%d"), ed.strftime("%Y/%m/%d")
)
msg = "Exchange 2013 re-created a meeting that was missing from your calendar."

print("from", st, "to", ed)

# ソートして定期的な予定を分解してからフィルタかける
for store in stores:
    cal = store.GetDefaultFolder(9)
    appointments = cal.Items
    appointments.sort("[Start]")
    appointments.IncludeRecurrences = True
    restricted = appointments.Restrict(filterStr)
    print("store:",store.DisplayName)
    counter = 0
    # Outlook側の予定アイテムにアクセス
    events = []
    for item in restricted:
        #for item in appointments:
        try:
            # print('\t',item.start,item.subject)
            if msg in item.body:
                print('\t\t',item.start,item.subject,item.body[:23])
                events.append(item)
        except Exception as ex:
            print(ex)
    if len(events)>0:
        # print("delete")
        for item in events:
            # print(item.start,item.subject)
            item.delete()
print("done.")

from 2021-08-25 to 2021-10-27
store: Jiro.Kogi@nttdata.com
store: Jiro.Kogi@jp.nttdata.com
store: オンライン アーカイブ - Jiro.Kogi@jp.nttdata.com
store: 2020
done.
